# Cleaning Covid data for Mapbox: provinces

**Background**: We use Covid-19 cases data in the Philippines from data from the health department and shapefile processed through geopandas to create an interactive map. 

**Tools**: pandas, geopandas, Mapbox

As of May 6, 2023

# Do your imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
pd.set_option('display.max_columns', None)

# Read your CSV

In [2]:
df= pd.read_csv('../data/provinces-cases.csv')
df

,province,cases
0,Lanao Del Sur,8847
1,Maguindanao,8408
2,Cotabato City (Not A Province),6830
3,Basilan,1915
4,Sulu,1738
...,...,...
101,NaN,2660
102,1,166570
103,"NCR, Second District",497360
104,"NCR, Third District",199003


# Cleaning the data

## Lowercase column headers

In [3]:
df.columns = df.columns.str.lower()
df.head()

,province,cases
0,Lanao Del Sur,8847
1,Maguindanao,8408
2,Cotabato City (Not A Province),6830
3,Basilan,1915
4,Sulu,1738


## Clean provincial names

This is so they would match the names in the shapefile for merging later.

In [4]:
df.province = df.province.str.replace("City Of Isabela (Not A Province)", "City of Isabela", regex=False)
df.province = df.province.str.replace("Cotabato (North Cotabato)", "Cotabato", regex=False)
df.province = df.province.str.replace("Samar (Western Samar)", "Samar", regex=False)
df.province = df.province.str.replace("Cotabato City (Not A Province)", "Cotabato City", regex=False)
df.province = df.province.str.replace("Del", "del", regex=False)
df.province = df.province.str.replace("De", "de", regex=False)
df.province = df.province.str.replace("1", "NCR, City of Manila, First District", regex=False)

## Rename columns

This is again to match the shapefile column containing the provinces' names which is 'adm2_en'. We are also renaming the column containing the Covid-19 cases tally.

In [5]:
df= df.rename(columns={"province": "adm2_en"})
df

,adm2_en,cases
0,Lanao del Sur,8847
1,Maguindanao,8408
2,Cotabato City,6830
3,Basilan,1915
4,Sulu,1738
...,...,...
101,NaN,2660
102,"NCR, City of Manila, First District",166570
103,"NCR, Second District",497360
104,"NCR, Third District",199003


# Geopandas

## Read through file

In [6]:
provinces = gpd.read_file('provinces.zip')
provinces

,Shape_Leng,Shape_Area,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,2.640967,0.334223,Abra,PH140100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946..."
1,3.674955,0.220065,Agusan del Norte,PH160200000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9..."
2,5.222636,0.693968,Agusan del Sur,PH160300000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446..."
3,4.626091,0.139664,Aklan,PH060400000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ..."
4,6.507665,0.205939,Albay,PH050500000,None,None,None,Region V,PH050000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,15.971439,0.094186,Tawi-Tawi,PH157000000,None,None,None,Bangsamoro Autonomous Region in Muslim Mindanao,PH150000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4..."
83,5.329770,0.313705,Zambales,PH037100000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((120.11687 14.76309, 120.11689 ..."
84,8.170921,0.515482,Zamboanga del Norte,PH097200000,None,None,None,Region IX,PH090000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.09474 7.53104, 122.09482 7..."
85,11.811347,0.439807,Zamboanga del Sur,PH097300000,None,None,None,Region IX,PH090000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.05710 6.87274, 122.05724 6..."


## Merge data 

In [7]:
provinces.columns = provinces.columns.str.lower()
provinces

,shape_leng,shape_area,adm2_en,adm2_pcode,adm2_ref,adm2alt1en,adm2alt2en,adm1_en,adm1_pcode,adm0_en,adm0_pcode,date,validon,validto,geometry
0,2.640967,0.334223,Abra,PH140100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946..."
1,3.674955,0.220065,Agusan del Norte,PH160200000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9..."
2,5.222636,0.693968,Agusan del Sur,PH160300000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446..."
3,4.626091,0.139664,Aklan,PH060400000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ..."
4,6.507665,0.205939,Albay,PH050500000,None,None,None,Region V,PH050000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,15.971439,0.094186,Tawi-Tawi,PH157000000,None,None,None,Bangsamoro Autonomous Region in Muslim Mindanao,PH150000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4..."
83,5.329770,0.313705,Zambales,PH037100000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((120.11687 14.76309, 120.11689 ..."
84,8.170921,0.515482,Zamboanga del Norte,PH097200000,None,None,None,Region IX,PH090000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.09474 7.53104, 122.09482 7..."
85,11.811347,0.439807,Zamboanga del Sur,PH097300000,None,None,None,Region IX,PH090000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.05710 6.87274, 122.05724 6..."


## Rename 'Compostela Valley' to 'Davao de Oro'

The name change was by virtue of a law.

In [8]:
provinces.adm2_en = provinces.adm2_en.str.replace("Compostela Valley", "Davao de Oro", regex=False)

In [9]:
provinces_cases = provinces.merge(df, on='adm2_en')
provinces_cases

,shape_leng,shape_area,adm2_en,adm2_pcode,adm2_ref,adm2alt1en,adm2alt2en,adm1_en,adm1_pcode,adm0_en,adm0_pcode,date,validon,validto,geometry,cases
0,2.640967,0.334223,Abra,PH140100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6467
1,3.674955,0.220065,Agusan del Norte,PH160200000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23830
2,5.222636,0.693968,Agusan del Sur,PH160300000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",18283
3,4.626091,0.139664,Aklan,PH060400000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17463
4,6.507665,0.205939,Albay,PH050500000,None,None,None,Region V,PH050000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,15.971439,0.094186,Tawi-Tawi,PH157000000,None,None,None,Bangsamoro Autonomous Region in Muslim Mindanao,PH150000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",786
83,5.329770,0.313705,Zambales,PH037100000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((120.11687 14.76309, 120.11689 ...",24063
84,8.170921,0.515482,Zamboanga del Norte,PH097200000,None,None,None,Region IX,PH090000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.09474 7.53104, 122.09482 7...",13634
85,11.811347,0.439807,Zamboanga del Sur,PH097300000,None,None,None,Region IX,PH090000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.05710 6.87274, 122.05724 6...",49538


## Drop unnecessary columns

In [10]:
provinces_cases = provinces_cases.drop(['adm2_ref', 'adm2_pcode','adm2alt1en', 'adm2alt2en', 'adm0_en', 'shape_leng', 'shape_area', 'adm0_pcode', 'adm1_pcode', 'date', 'validon', 'validto'], axis=1)
provinces_cases

,adm2_en,adm1_en,geometry,cases
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6467
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23830
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",18283
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17463
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15910
...,...,...,...,...
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",786
83,Zambales,Region III,"MULTIPOLYGON (((120.11687 14.76309, 120.11689 ...",24063
84,Zamboanga del Norte,Region IX,"MULTIPOLYGON (((122.09474 7.53104, 122.09482 7...",13634
85,Zamboanga del Sur,Region IX,"MULTIPOLYGON (((122.05710 6.87274, 122.05724 6...",49538


In [11]:
provinces_cases= provinces_cases.rename(columns={"adm2_en": "province"})

## Read and merge with population data

In [12]:
df2 = pd.read_excel('population.xlsx', sheet_name="province")
df2.head()

,province,population
0,"NCR, City of Manila, First District",1846513
1,"NCR, Second District",4771371
2,"NCR, Third District",3004627
3,"NCR, Fourth District",3861951
4,Abra,250985


In [13]:
provinces_final = provinces_cases.merge(df2, on='province')
provinces_final.head()

,province,adm1_en,geometry,cases,population
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6467,250985
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23830,760413
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",18283,739367
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17463,615475
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15910,1374768


## Compute for population ratio

We do this by dividing the number of Covid-19 cases to total population per province and then multiply by 100,000. That would give us cases per 100,000 people in the area.

In [14]:
provinces_final ['case_per_pop'] = provinces_final.cases / provinces_final.population * 100000
provinces_final = provinces_final.round(1)
provinces_final.head()

,province,adm1_en,geometry,cases,population,case_per_pop
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6467,250985,2576.6
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23830,760413,3133.8
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",18283,739367,2472.8
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17463,615475,2837.3
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15910,1374768,1157.3


## Create bins for cases

The bins will allow us to categorize the number of cases, necessary for mapping later.

In [15]:
provinces_final['percentiles'] = pd.cut(np.array(provinces_final['case_per_pop']),
       [0, 1001, 2001, 3001, 4001, 5001, 6001, 7001, 8001, 9001, 10001, 11001,12000], labels=["0-1000", "1001-2000", "2001-3000", "3001-4000", "4001-5000", "5001-6000", "6001-7000", "7001-8000", "8001-9000", "9001-10000", "10001-11000", "11001-12000"])


In [16]:
provinces_final

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6467,250985,2576.6,2001-3000
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23830,760413,3133.8,3001-4000
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",18283,739367,2472.8,2001-3000
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17463,615475,2837.3,2001-3000
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15910,1374768,1157.3,1001-2000
...,...,...,...,...,...,...,...
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",786,440276,178.5,0-1000
83,Zambales,Region III,"MULTIPOLYGON (((120.11687 14.76309, 120.11689 ...",24063,909832,2644.8,2001-3000
84,Zamboanga del Norte,Region IX,"MULTIPOLYGON (((122.09474 7.53104, 122.09482 7...",13634,1047455,1301.6,1001-2000
85,Zamboanga del Sur,Region IX,"MULTIPOLYGON (((122.05710 6.87274, 122.05724 6...",49538,2027902,2442.8,2001-3000


In [23]:
provinces_final.sort_values('case_per_pop', ascending=False)

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
54,"NCR, Fourth District",National Capital Region,"POLYGON ((121.03468 14.56725, 121.03521 14.567...",448721,3861951,11619.0,11001-12000
55,"NCR, Second District",National Capital Region,"POLYGON ((121.13497 14.77673, 121.13447 14.776...",497360,4771371,10423.8,10001-11000
12,Benguet,Cordillera Administrative Region,"POLYGON ((120.75882 16.92650, 120.77062 16.921...",77524,827041,9373.7,9001-10000
53,"NCR, City of Manila, First District",National Capital Region,"MULTIPOLYGON (((120.94997 14.63625, 120.94997 ...",166570,1846513,9020.8,9001-10000
6,Apayao,Cordillera Administrative Region,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9634,124366,7746.5,7001-8000
...,...,...,...,...,...,...,...
49,Masbate,Region V,"MULTIPOLYGON (((122.98823 11.73079, 122.98824 ...",6678,908920,734.7,0-1000
47,Maguindanao,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((124.07624 7.37644, 124.07630 7...",8408,1342179,626.4,0-1000
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6...",1915,426207,449.3,0-1000
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",786,440276,178.5,0-1000


In [17]:
provinces_final.dtypes

province          object
adm1_en           object
geometry        geometry
cases              int64
population         int64
case_per_pop     float64
percentiles     category
dtype: object

**Additional step**: Convert the contents of the percentiles into string. Not doing so will not be read by the GEOJSON file.

In [18]:
provinces_final.percentiles = provinces_final.percentiles.astype(str)

In [19]:
provinces_final

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6467,250985,2576.6,2001-3000
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23830,760413,3133.8,3001-4000
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",18283,739367,2472.8,2001-3000
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17463,615475,2837.3,2001-3000
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15910,1374768,1157.3,1001-2000
...,...,...,...,...,...,...,...
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",786,440276,178.5,0-1000
83,Zambales,Region III,"MULTIPOLYGON (((120.11687 14.76309, 120.11689 ...",24063,909832,2644.8,2001-3000
84,Zamboanga del Norte,Region IX,"MULTIPOLYGON (((122.09474 7.53104, 122.09482 7...",13634,1047455,1301.6,1001-2000
85,Zamboanga del Sur,Region IX,"MULTIPOLYGON (((122.05710 6.87274, 122.05724 6...",49538,2027902,2442.8,2001-3000


In [20]:
provinces_final.sort_values('percentiles', ascending=False)

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
53,"NCR, City of Manila, First District",National Capital Region,"MULTIPOLYGON (((120.94997 14.63625, 120.94997 ...",166570,1846513,9020.8,9001-10000
12,Benguet,Cordillera Administrative Region,"POLYGON ((120.75882 16.92650, 120.77062 16.921...",77524,827041,9373.7,9001-10000
6,Apayao,Cordillera Administrative Region,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9634,124366,7746.5,7001-8000
52,Mountain Province,Cordillera Administrative Region,"POLYGON ((121.48738 17.28923, 121.48795 17.289...",11422,158200,7220.0,7001-8000
56,"NCR, Third District",National Capital Region,"POLYGON ((121.03842 14.78525, 121.03876 14.785...",199003,3004627,6623.2,6001-7000
...,...,...,...,...,...,...,...
47,Maguindanao,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((124.07624 7.37644, 124.07630 7...",8408,1342179,626.4,0-1000
18,Camarines Norte,Region V,"MULTIPOLYGON (((123.10510 14.04671, 123.10559 ...",5511,629699,875.2,0-1000
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6...",1915,426207,449.3,0-1000
59,Northern Samar,Region VIII,"MULTIPOLYGON (((124.09920 12.29055, 124.09920 ...",5244,639186,820.4,0-1000


# Save as GEOJSON file

In [21]:
provinces_final.to_file('provinces_cases.geojson', driver='GeoJSON')

# Simplified file

So we were successful in combining geometry files with our dataset, but the file is too big. We, therefore, use [mapshaper](https://mapshaper.org/) to simplify the precision of the map so that we have a smaller map size.

Below is the simplified json file. 

In [24]:
simplified = gpd.read_file('provinces_cases.json')
simplified

,province,adm1_en,cases,population,case_per_pop,percentiles,geometry
0,Abra,Cordillera Administrative Region,6467,250985,2576.6,2001-3000,"POLYGON ((121.11067 17.68489, 121.10883 17.739..."
1,Agusan del Norte,Region XIII,23830,760413,3133.8,3001-4000,"MULTIPOLYGON (((125.74862 9.32604, 125.74822 9..."
2,Agusan del Sur,Region XIII,18283,739367,2472.8,2001-3000,"POLYGON ((126.22779 8.00019, 126.22786 8.00055..."
3,Aklan,Region VI,17463,615475,2837.3,2001-3000,"MULTIPOLYGON (((122.42966 11.59990, 122.42917 ..."
4,Albay,Region V,15910,1374768,1157.3,1001-2000,"MULTIPOLYGON (((124.20992 13.16871, 124.20944 ..."
...,...,...,...,...,...,...,...
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,786,440276,178.5,0-1000,"MULTIPOLYGON (((119.46876 4.59360, 119.46807 4..."
83,Zambales,Region III,24063,909832,2644.8,2001-3000,"MULTIPOLYGON (((120.11687 14.76309, 120.11656 ..."
84,Zamboanga del Norte,Region IX,13634,1047455,1301.6,1001-2000,"MULTIPOLYGON (((122.09474 7.53104, 122.09436 7..."
85,Zamboanga del Sur,Region IX,49538,2027902,2442.8,2001-3000,"MULTIPOLYGON (((122.05710 6.87274, 122.05624 6..."


## Convert to GEOJSON

In [25]:
simplified.to_file('simplified_provinces.geojson', driver='GeoJSON')

## Fix CSV to upload to website

In [26]:
simplified = simplified.drop(['geometry', 'percentiles', 'population'], axis=1)
simplified

,province,adm1_en,cases,case_per_pop
0,Abra,Cordillera Administrative Region,6467,2576.6
1,Agusan del Norte,Region XIII,23830,3133.8
2,Agusan del Sur,Region XIII,18283,2472.8
3,Aklan,Region VI,17463,2837.3
4,Albay,Region V,15910,1157.3
...,...,...,...,...
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,786,178.5
83,Zambales,Region III,24063,2644.8
84,Zamboanga del Norte,Region IX,13634,1301.6
85,Zamboanga del Sur,Region IX,49538,2442.8


In [27]:
simplified.columns = ['Province', 'Region', 'Covid-19 cases', 'Case per 100,000 population']
simplified

,Province,Region,Covid-19 cases,Case per population
0,Abra,Cordillera Administrative Region,6467,2576.6
1,Agusan del Norte,Region XIII,23830,3133.8
2,Agusan del Sur,Region XIII,18283,2472.8
3,Aklan,Region VI,17463,2837.3
4,Albay,Region V,15910,1157.3
...,...,...,...,...
82,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindanao,786,178.5
83,Zambales,Region III,24063,2644.8
84,Zamboanga del Norte,Region IX,13634,1301.6
85,Zamboanga del Sur,Region IX,49538,2442.8


In [28]:
simplified.to_csv('../data/provinces-cases.csv', index=False)